In [3]:
import pyart
from matplotlib import pyplot as plt
import numpy as np
import numpy.ma as ma
from numpy import dtype
import os
from netCDF4 import Dataset,date2num,num2date
import metpy.calc
from metpy.units import units
import warnings
warnings.filterwarnings('ignore')
from wradlib.trafo import r_to_depth
import datetime as dt
import pandas as pd
from datetime import datetime
from scipy.stats import mstats
import numpy.ma as ma

In [4]:
linkRAW="C:/Users/Admin/Desktop/test_py_WRADLIB/"
linkOUT="C:/Users/Admin/Desktop/test_py_WRADLIB/"

In [5]:
class UF:
    def __init__(self,radar,shape_grid,lat_0, lon_0):
        self.radar = radar
        self.shape_grid = shape_grid
        self.lat_0 = lat_0
        self.lon_0 = lon_0
    def calculate_attenuation_zphi(self):
        fixed_fzl=3000
        spec_at, pia_dict, cor_z, spec_diff_at, pida_dict, cor_zdr= pyart.correct.calculate_attenuation_zphi(
            self.radar,
            refl_field='reflectivity', 
            phidp_field= 'differential_phase',
            zdr_field='differential_reflectivity',
            temp_ref='fixed_fzl')
        return cor_z
    def noise_rejection(self):
        range_bins=self.radar.range['data']
        ref_last_bins=self.radar.fields['reflectivity']['data'][:,-5:]
        Z_f=mstats.mode(ref_last_bins,axis=None)[0][0]
        Z_s=pyart.retrieve.compute_noisedBZ(360, noisedBZ_val=Z_f, _range=range_bins, ref_dist=300., noise_field='noise_dbz')
        Z_splus4=Z_s['data']+4
        conditions_masked=ma.masked_less(self.radar.fields['reflectivity']['data'],Z_splus4)
        self.radar.add_field_like('reflectivity', 'reflectivity_cor_noise',conditions_masked,replace_existing=True)
    def convert_grid(self):
        grid = pyart.map.grid_from_radars(
            self.radar,
            grid_shape=self.shape_grid, #Number of points in the grid (z, y, x)
            grid_limits=((0, 12000), (-300000, 300000), (-300000, 300000)), # min-max tuong duong z,y,x
            grid_origin = (self.lat_0, self.lon_0),
            fields=['rain_R_Z'])
        return grid
    def convert_lat_lon(self, grid):
        for i in range(self.shape_grid[1]):
            geog = pyart.core.cartesian_to_geographic_aeqd(grid.x["data"][i],grid.y["data"][i],self.lon_0, self.lat_0, R=6370997.0)
            grid.x["data"][i] = geog[0]
            grid.y["data"][i] = geog[1]
        return grid
    def convert_Z_to_R(self):
        rain_R_Z=pyart.retrieve.est_rain_rate_z(self.radar,alpha=0.0376, beta=0.6112,refl_field='reflectivity_cor_noise')
        rain_R_Z_mm=r_to_depth(rain_R_Z['data'],600)
        rain_R_Z['data']=rain_R_Z_mm
        rain_R_Z['units']='mm'
        self.radar.add_field('rain_R_Z', rain_R_Z)

In [6]:
def main(filein,shape_grid):
    radar = pyart.io.read_sigmet(filein)
    radar=radar.extract_sweeps([0]) # axtract sweep 0,5
    time_unit=radar.time['units']
    lat_0 = radar.latitude['data'][0]
    lon_0 = radar.longitude['data'][0]   
    UF_File = UF(radar, shape_grid, lat_0, lon_0)
    ref_cor_noise=UF_File.noise_rejection()
    convert_Z_to_R=UF_File.convert_Z_to_R()
    grid = UF_File.convert_grid()
    grid = UF_File.convert_lat_lon(grid)
    return grid,time_unit

In [7]:
shape_grid=(1,601,601)
pre_Z_R=np.zeros(shape_grid)
interval_time=[]
for name in os.listdir(linkRAW):
    filein = linkRAW + name
    grid,time_unit=main(filein,shape_grid)
    grid_pre=grid.fields['rain_R_Z']['data'] 
    pre_Z_R=np.append(pre_Z_R,grid_pre,axis=0)
    interval_time.append(time_unit)
    print(filein)

C:/Users/Admin/Desktop/test_py_WRADLIB/PHA210208200004.RAWMLLS


In [8]:
times_values=[]
for time in interval_time:
    format_string_time = "%Y-%m-%dT%H:%M"
    ti=time[14:-4]
    format_time=datetime.strptime(ti, format_string_time)
    times_values.append(format_time)

In [9]:
pre_Z_R=pre_Z_R[1:,:,:]
pre_Z_R=ma.masked_equal(pre_Z_R,0)
pre_Z_R.shape

(1, 601, 601)

In [10]:
fileout=linkOUT+name[0:13]
def create_nc(fileout, grid, pre_Z_R,times_values):
    file = Dataset(fileout+ ".nc",'w')
    file.title = "Netcdf radar file - Made by PYTHON"
    file.createDimension('longitude',601)
    file.createDimension('latitude',601)
    file.createDimension('time',None)
    times = file.createVariable('time',dtype('i8').char,('time',))
    lons = file.createVariable('longitude',dtype('f4').char,('longitude',))
    lats = file.createVariable('latitude',dtype('f4').char,('latitude',))
    lats.units = 'degrees_north'
    lats.standard_name = "Latitude"
    lats.long_name = "Latitude"
    lats.axis = "Y"
    lons.standard_name = "Longitude"
    lons.long_name = "Longitude"
    lons.axis = "X"
    lons.units = 'degrees_east'
    times.units = "minutes since 0001-01-01 00:00"
    times.calendar = "gregorian"
    times.standard_name = "time"
    times.axis = "T"
    lons[:] = grid.x["data"]
    lats[:] = grid.y["data"]
    times[:] =date2num(times_values,units=times.units,calendar=times.calendar)
    ref = file.createVariable('precipitation',dtype('f4').char,('time','latitude','longitude'))
    ref.units = 'mm'
    ref[:] =pre_Z_R
    file.close()

In [11]:
make_file=create_nc(fileout, grid, pre_Z_R,times_values)

In [14]:
data=Dataset('C:/Users/Admin/Desktop/test_py_WRADLIB/PHA2102082000.nc','r')

In [17]:
data.variables['precipitation']

<class 'netCDF4._netCDF4.Variable'>
float32 precipitation(time, latitude, longitude)
    units: mm
unlimited dimensions: time
current shape = (1, 601, 601)
filling on, default _FillValue of 9.969209968386869e+36 used